## Exploring the SQUADv2 dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxx'

#### Install the autolabel library

In [2]:
!pip3 install 'refuel-autolabel[openai]'

#### Download the dataset

In [3]:
from autolabel import get_data

get_data('squad_v2')

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100% [........................................................] 177008 / 177008

100% [......................................................] 1786563 / 1786563

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

In [5]:
import json

from autolabel import LabelingAgent

In [10]:
# load the config
with open('config_squad_v2.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `question_answering` (since it's a question answering task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at answering questions based on wikipedia articles` (how we describe the task to the LLM)
* `prompt.few_shot_num`: 3 (how many labeled examples to provide to the LLM)

In [11]:
config

{'task_name': 'OpenbookQAWikipedia',
 'task_type': 'question_answering',
 'dataset': {'label_column': 'answer',
  'explanation_column': 'explanation',
  'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'You are an expert at answering questions based on wikipedia articles. Your job is to answer the following questions using the context provided with the question. Use the context to answer the question - the answer is a continuous span of words from the context.\n',
  'output_guidelines': 'Your answer will consist of an explanation, followed by the correct answer. The last line of the response should always be is JSON format with one key: {"label": "the correct answer"}.\n If the question cannot be answered using the context and the context alone without any outside knowledge, the question is unanswerable. If the question is unanswerable, return the answer as {"label": "unanswerable"}\n',
  'few_shot_examples': 'seed.csv',
  'fe

In [22]:
# generate explanations for chain of thought
seed_with_explanations = agent.generate_explanations('seed.csv')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

2023-06-14 15:59:48 openai INFO: error_code=None error_message='That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 87d1cac9c00b5c4994a0e5fe446a2087 in your message.)' error_param=None error_type=server_error message='OpenAI API error received' stream_error=False
2023-06-14 15:59:48 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 87d1cac9c00b5c4994a0e5fe446a2087 in your message.).


In [23]:
# Use seedset with explanations in config
config['prompt']['few_shot_examples'] = seed_with_explanations

In [24]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [25]:
agent.plan('test.csv')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $8.7028 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0044 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at answering questions based on wikipedia articles. Your job is to answer the following questions using the context provided with the question. Use the context to answer the question - the answer is a continuous span of words from the context.


Your answer will consist of an explanation, followed by the correct answer. The last line of the response should always be is JSON format with one key: {"label": "the correct answer"}.
 If the question cannot be answered using the context and the context alone without any outside knowledge, the question is unanswerable. If the question is unanswerable, return the answer as {"label": "unanswerable"}


Some examples with their output answers are provided below:

Context: The final major evolution of the steam engine design was the use of steam turbines starting in the late part of the 19th century. Steam turbines are generally more efficient than reciprocating piston type steam engines (for outputs above several hundred horsepow

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [26]:
labels, df, metrics_list = agent.run('test.csv', max_items=100)

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Metric: auroc: 0.4433
Actual Cost: 0.2522


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.449  │ 100     │ -inf      │ 0.06     │ 1.0             │
│ 0.6667 │ 1       │ 0.9043    │ 0.0      │ 0.01            │
│ 0.8333 │ 2       │ 0.8987    │ 0.5      │ 0.02            │
│ 0.7981 │ 5       │ 0.8838    │ 0.2      │ 0.05            │
│ 0.8317 │ 6       │ 0.8838    │ 0.3333   │ 0.06            │
│ 0.4774 │ 65      │ 0.7543    │ 0.0308   │ 0.65            │
│ 0.4853 │ 66      │ 0.7528    │ 0.0455   │ 0.66            │
│ 0.4932 │ 69      │ 0.7481    │ 0.0435   │ 0.69            │
│ 0.5004 │ 70      │ 0.7472    │ 0.0571   │ 0.7             │
│ 0.4436 │ 91      │ 0.6882    │ 0.044    │ 0.91            │
│ 0.4497 │ 92      │ 0.683     │ 0.0543   │ 0.92            │
│ 0.4479 │ 98      │ 0.651     │ 0.051    │ 0.98            │
│ 0.4535 │ 99      │ 0.6394    │ 0.0606   │ 0.99            │
│ 0.449  │ 100     │ 0.6066    │ 0.06     │ 1.0             │
└────────┴─────────┴───────────┴──────────┴─────────────────┘

Total number of failures: 0

We are at 94% accuracy when labeling the first 100 examples. Let's see if we can use confidence scores to improve accuracy further by removing the less confident examples from our labeled set.

### Compute confidence scores


In [27]:
# Start computing confidence scores (using Refuel's LLMs)
os.environ['REFUEL_API_KEY'] = 'xxxxxxxxxxxxxxxxx'

In [28]:
config["model"]["compute_confidence"] = True

In [29]:
agent = LabelingAgent(config=config)

In [30]:
agent.plan('test.csv')

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $8.7028 │
│ Number of Examples       │ 2000    │
│ Average cost per example │ $0.0044 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at answering questions based on wikipedia articles. Your job is to answer the following questions using the context provided with the question. Use the context to answer the question - the answer is a continuous span of words from the context.


Your answer will consist of an explanation, followed by the correct answer. The last line of the response should always be is JSON format with one key: {"label": "the correct answer"}.
 If the question cannot be answered using the context and the context alone without any outside knowledge, the question is unanswerable. If the question is unanswerable, return the answer as {"label": "unanswerable"}


Some examples with their output answers are provided below:

Context: The final major evolution of the steam engine design was the use of steam turbines starting in the late part of the 19th century. Steam turbines are generally more efficient than reciprocating piston type steam engines (for outputs above several hundred horsepow

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [31]:
labels, df, metrics_list = agent.run('test.csv', max_items=100)

2023-06-14 16:18:10 autolabel.labeler INFO: Task run already exists.


There is an existing task with following details: id='1775062444' created_at=datetime.datetime(2023, 6, 14, 16, 6, 
51, 951229) task_id='d1db10ad8148c126df439c7053a0abfd' dataset_id='d31f8a9decf338c2d8efea27263a0fe3' 
current_index=100 output_file='test_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None metrics=None

Evaluating the existing task...

Metric: auroc: 0.4432624113475177


┏━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ f1     ┃ support ┃ threshold ┃ accuracy ┃ completion_rate ┃
┡━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.449  │ 100     │ -inf      │ 0.06     │ 1.0             │
│ 0.6667 │ 1       │ 0.9043    │ 0.0      │ 0.01            │
│ 0.8333 │ 2       │ 0.8987    │ 0.5      │ 0.02            │
│ 0.7981 │ 5       │ 0.8838    │ 0.2      │ 0.05            │
│ 0.8317 │ 6       │ 0.8838    │ 0.3333   │ 0.06            │
│ 0.4774 │ 65      │ 0.7543    │ 0.0308   │ 0.65            │
│ 0.4853 │ 66      │ 0.7528    │ 0.0455   │ 0.66            │
│ 0.4932 │ 69      │ 0.7481    │ 0.0435   │ 0.69            │
│ 0.5004 │ 70      │ 0.7472    │ 0.0571   │ 0.7             │
│ 0.4436 │ 91      │ 0.6882    │ 0.044    │ 0.91            │
│ 0.4497 │ 92      │ 0.683     │ 0.0543   │ 0.92            │
│ 0.4479 │ 98      │ 0.651     │ 0.051    │ 0.98            │
│ 0.4535 │ 99      │ 0.6394    │ 0.0606   │ 0.99            │
│ 0.449  │ 100     │ 0.6066    │ 0.06     │ 1.0             │
└────────┴─────────┴───────────┴──────────┴─────────────────┘

...

Do you want to resume the task? [y/n]:

n


Deleted the existing task and starting a new one...

/opt/homebrew/lib/python3.10/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

KeyboardInterrupt: 